# Biomass composition ➟  biomass reaction
This notebook converts biomass composition measurements into biomass equations for various R. opacus strains growing on phenol and glucose.

### Method
<ol>
<li>Load biomass compositions for different strains and conditions</li>
<li>Calculate the macromolecule breakdown for the bacillus subtilis growth reaction built into CarveMe</li>
<li>For each macromolecule, calculate the ratio of R.opacus concentration : Bacillus subtilis concentration</li>
<li>Define phenol and glucose R.opacus biomass reactions by finding the coeffient for each precursor by multipling the B. subtilis coefficient by the macromolecule scaling factor</li>
</ol>

### Load dependencies

In [1]:
import cobra
import pandas as pd

### Get biomass composition data (.csv import is a temporary. In the future this data will be from EDD)

In [2]:
biomass_comp_filename = '../../EDD_Ropacus_Multiomics/EDD_Ropacus_Multiomics_BiomassComp.csv'
biomass_comp_df = pd.read_csv(biomass_comp_filename)

# remove blank columns
cols_to_drop = [col for col in biomass_comp_df.columns if 'Unnamed' in col]
biomass_comp_df.drop(cols_to_drop, inplace=True, axis=1)

biomass_comp_df.head(10)
# biomass_comp_df[(biomass_comp_df['Line Name']  == 'RoWT_PHE-R2') | (biomass_comp_df['Line Name'] == 'RoWT_GLC-R2')]

,Line Name,Measurement Type,Time (hrs),Value,Units
0,RoWT_PHE-R2,Lipid Content,25.5,14.08,%
1,RoWT_PHE-R2,Carbohydrate Content,25.5,17.21,%
2,RoWT_PHE-R2,Protein Content,25.5,38.84,%
3,RoP1_L-R1,Lipid Content,25.5,16.44,%
4,RoP1_L-R1,Carbohydrate Content,25.5,12.47,%


### Load model and pull up original biomass equation (from Bacillus subtilis)

In [3]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated_with_phenol.xml")
model

Name,ropacus_annotated_curated
Memory address,0x07f31cf580610
Number of metabolites,1583
Number of reactions,2383
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


In [27]:
# biomass_comps = pd.DataFrame({'B. subtilis' : })
b_subtiltis_biomass_reaction = model.reactions.get_by_id('Growth').reaction

consumed_string, produced_string = b_subtiltis_biomass_reaction.split(' --> ')

consumed_list = consumed_string.split(' + ')
produced_list = produced_string.split(' + ')

consumed_dict = {met.split(' ')[1] : -1* float(met.split(' ')[0]) for met in consumed_list}
produced_dict = {met.split(' ')[1] : float(met.split(' ')[0]) for met in produced_list}

def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

biomass_comps = pd.DataFrame.from_dict(Merge(consumed_dict, produced_dict), orient='index', columns=['B. subilis'])
biomass_comps


,B. subilis
10fthf_c,-0.000216
ala__L_c,-0.498716
amet_c,-0.000216
arg__L_c,-0.287170
asn__L_c,-0.234029
asp__L_c,-0.234029
atp_c,-52.547151
ca2_c,-0.005053
cl_c,-0.005053
coa_c,-0.000559


### Load dataframes with metabolite information
These csv files contain information about the reactants and products.

In [5]:
substrate_df = pd.read_csv('../biomass_equation_data/subtilis_biomass_reactants.csv')
substrate_df['coefficient'] = -1*substrate_df['coefficient'] # this is needed because these metabolites are consumed in biomass reaction

product_df = pd.read_csv('../biomass_equation_data/subtilis_biomass_products.csv')

all_mets_df = pd.concat([substrate_df, product_df], sort=False)
all_mets_df

,coefficient,name,abbreviation,category,molecular_weight
0,-0.000216,10-Formyltetrahydrofolate,10fthf_c,carbon carrier,471.430
1,-0.498716,L-Alanine,ala__L_c,protein,89.094
2,-0.000216,S-Adenosyl-L-methionine,amet_c,carbon carrier,399.446
3,-0.287170,L-Arginine,arg__L_c,protein,175.212
4,-0.234029,L-Asparagine,asn__L_c,protein,132.119
5,-0.234029,L-Aspartate,asp__L_c,protein,132.095
6,-52.547151,ATP,atp_c,energy molecule,503.150
7,-0.005053,Calcium,ca2_c,salt,40.078
8,-0.005053,Chloride,cl_c,salt,35.450
9,-0.000559,CoenzymeA,coa_c,carbon carrier,763.502


### Calculate the macromolecule distribution in the Subtilis biomass equation

In [6]:
subtilis_mg_protein = 0
subtilis_mg_lipid = 0
subtilis_mg_carbohydrate = 0

for met in list(substrate_df['abbreviation']):
    met_category = list(substrate_df[substrate_df['abbreviation'] == met]['category'])[0]
    met_molar_mass = list(substrate_df[substrate_df['abbreviation'] == met]['molecular_weight'])[0]
    met_coefficient = abs(list(substrate_df[substrate_df['abbreviation'] == met]['coefficient'])[0])
    
    if met_category == 'protein':
        subtilis_mg_protein += met_molar_mass*met_coefficient
    if met_category == 'lipid':
        subtilis_mg_lipid += met_molar_mass*met_coefficient
    if met_category == 'carbohydrate':
        subtilis_mg_carbohydrate += met_molar_mass*met_coefficient

print(f'subtiltis mg of protein / g dry cell weight: {subtilis_mg_protein:.2f} mg')
print(f'subtiltis mg of lipid / g dry cell weight: {subtilis_mg_lipid:.2f} mg')
print(f'subtiltis mg of carbohydrate / g dry cell weight: {subtilis_mg_carbohydrate:.2f} mg')

subtiltis mg of protein / g dry cell weight: 656.05 mg
subtiltis mg of lipid / g dry cell weight: 175.24 mg
subtiltis mg of carbohydrate / g dry cell weight: 9.62 mg


### Define function to take in a strain and measurement type and returns a value

In [7]:
def strain_measurement_value(strain, measurement_type):
    return float(biomass_comp_df[(biomass_comp_df['Line Name']  == strain) & (biomass_comp_df['Measurement Type'] == measurement_type)]['Value'])

strain_measurement_value('RoWT_PHE-R2', 'Protein Content')

38.84

### Add Phenol Biomass Equation

Calculate the scaling factor for each macromolecule

In [8]:
ropacus_phenol_mg_protein = 10 * strain_measurement_value('RoWT_PHE-R2', 'Protein Content')
ropacus_phenol_mg_lipid = 10 * strain_measurement_value('RoWT_PHE-R2', 'Lipid Content')
ropacus_phenol_mg_carbohydrate = 10 * strain_measurement_value('RoWT_PHE-R2', 'Carbohydrate Content')

lipid_multiplier = ropacus_phenol_mg_protein / subtilis_mg_lipid
protein_multiplier = ropacus_phenol_mg_lipid/ subtilis_mg_protein
carbohydrate_multiplier = ropacus_phenol_mg_carbohydrate/ subtilis_mg_carbohydrate

print(f'R. opacus phenol mg of protein / g dry cell weight: {ropacus_phenol_mg_protein:.2f} mg')
print(f'R. opacus phenol mg of lipid / g dry cell weight: {ropacus_phenol_mg_lipid:.2f} mg')
print(f'R. opacus phenol mg of carbohydrate / g dry cell weight: {ropacus_phenol_mg_carbohydrate:.2f} mg\n')

print(f'The lipid content is {lipid_multiplier:.3f} times higher in R. opacus with phenol than in B. Subtilis.')
print(f'The protein content is {protein_multiplier:.3f} times higher in R. opacus with phenol than in B. Subtilis.')
print(f'The carbohydrate content is {carbohydrate_multiplier:.3f} times higher in R. opacus with phenol than in B. Subtilis.')

R. opacus phenol mg of protein / g dry cell weight: 388.40 mg
R. opacus phenol mg of lipid / g dry cell weight: 140.80 mg
R. opacus phenol mg of carbohydrate / g dry cell weight: 172.10 mg

The lipid content is 2.216 times higher in R. opacus with phenol than in B. Subtilis.
The protein content is 0.215 times higher in R. opacus with phenol than in B. Subtilis.
The carbohydrate content is 17.887 times higher in R. opacus with phenol than in B. Subtilis.


Create a new reaction for phenol biomass

In [9]:
phenol_growth_equation = cobra.Reaction()
phenol_growth_equation.name = 'Phenol biomass reaction'
phenol_growth_equation.id = 'Growth_Phenol'

metabolites_to_add = {}

# for met in list(substrate_df['abbreviation']):
for metabolite in list(model.reactions.get_by_id('Growth').metabolites):
    met = metabolite.id
    met_category = list(all_mets_df[all_mets_df['abbreviation'] == met]['category'])[0]
    met_subtilis_coefficient = list(all_mets_df[all_mets_df['abbreviation'] == met]['coefficient'])[0]
    metabolite = model.metabolites.get_by_id(met)
    
    if met_category == 'protein':
        met_ropacus_coefficient = protein_multiplier*met_subtilis_coefficient
    elif met_category == 'lipid':
        met_ropacus_coefficient = lipid_multiplier*met_subtilis_coefficient
    elif met_category == 'carbohydrate':
        met_ropacus_coefficient = carbohydrate_multiplier*met_subtilis_coefficient
    else:
        met_ropacus_coefficient = met_subtilis_coefficient
        
    metabolites_to_add[metabolite] = met_ropacus_coefficient
    
phenol_growth_equation.add_metabolites(metabolites_to_add)
model.add_reactions([phenol_growth_equation])

model.reactions.get_by_id('Growth_Phenol').reaction

'0.00021600000000000002 10fthf_c + 0.10703270532982996 ala__L_c + 0.00021600000000000002 amet_c + 0.06163143350036347 arg__L_c + 0.05022649563205266 asn__L_c + 0.05022649563205266 asp__L_c + 52.547151 atp_c + 0.005053 ca2_c + 0.005053 cl_c + 0.000559 coa_c + 9.7e-05 cobalt2_c + 0.129616 ctp_c + 0.000688 cu2_c + 0.0190817717169301 cys__L_c + 0.025403 datp_c + 0.026229000000000002 dctp_c + 0.026229000000000002 dgtp_c + 0.025403 dttp_c + 0.00021600000000000002 fad_c + 0.006519 fe2_c + 0.00758 fe3_c + 0.05483216670813233 gln__L_c + 0.05483216670813233 glu__L_c + 0.12764962919993797 gly_c + 0.0021521013239706715 gtca1_45_BS_c + 0.0021521013239706715 gtca2_45_BS_c + 0.0021521013239706715 gtca3_45_BS_c + 0.208826 gtp_c + 47.184845 h2o_c + 0.019739571430265803 his__L_c + 0.060534742950560716 ile__L_c + 0.189503 k_c + 0.09387306258183474 leu__L_c + 0.00010860243550418423 lipo1_24_BS_c + 0.00010860243550418423 lipo2_24_BS_c + 0.00010860243550418423 lipo3_24_BS_c + 0.00010860243550418423 lipo4_24

### Test phenol biomass equation

Calculate breakdown of macromolecules in mg / g dry cell weight

In [10]:
# Code

### Add glucose biomass equation to model

Calculate the scaling factor for each macromolecule

In [11]:
ropacus_glucose_mg_protein = 10 * strain_measurement_value('RoWT_GLC-R2', 'Protein Content')
ropacus_glucose_mg_lipid = 10 * strain_measurement_value('RoWT_GLC-R2', 'Lipid Content')
ropacus_glucose_mg_carbohydrate = 10 * strain_measurement_value('RoWT_GLC-R2', 'Carbohydrate Content')

lipid_multiplier = ropacus_glucose_mg_protein / subtilis_mg_lipid
protein_multiplier = ropacus_glucose_mg_lipid/ subtilis_mg_protein
carbohydrate_multiplier = ropacus_glucose_mg_carbohydrate/ subtilis_mg_carbohydrate

print(f'R. opacus glucose mg of protein / g dry cell weight: {ropacus_glucose_mg_protein:.2f} mg')
print(f'R. opacus glucose mg of lipid / g dry cell weight: {ropacus_glucose_mg_lipid:.2f} mg')
print(f'R. opacus glucose mg of carbohydrate / g dry cell weight: {ropacus_glucose_mg_carbohydrate:.2f} mg\n')

print(f'The lipid content is {lipid_multiplier:.3f} times higher in R. opacus with glucose than in B. Subtilis.')
print(f'The protein content is {protein_multiplier:.3f} times higher in R. opacus with glucose than in B. Subtilis.')
print(f'The carbohydrate content is {carbohydrate_multiplier:.3f} times higher in R. opacus with glucose than in B. Subtilis.')

R. opacus glucose mg of protein / g dry cell weight: 240.40 mg
R. opacus glucose mg of lipid / g dry cell weight: 402.60 mg
R. opacus glucose mg of carbohydrate / g dry cell weight: 146.40 mg

The lipid content is 1.372 times higher in R. opacus with glucose than in B. Subtilis.
The protein content is 0.614 times higher in R. opacus with glucose than in B. Subtilis.
The carbohydrate content is 15.216 times higher in R. opacus with glucose than in B. Subtilis.


Create a new reaction for glucose biomass

In [12]:
glucose_growth_equation = cobra.Reaction()
glucose_growth_equation.name = 'glucose biomass reaction'
glucose_growth_equation.id = 'Growth_Glucose'

metabolites_to_add = {}

# for met in list(substrate_df['abbreviation']):
for metabolite in list(model.reactions.get_by_id('Growth').metabolites):
    met = metabolite.id
    met_category = list(all_mets_df[all_mets_df['abbreviation'] == met]['category'])[0]
    met_subtilis_coefficient = list(all_mets_df[all_mets_df['abbreviation'] == met]['coefficient'])[0]
    metabolite = model.metabolites.get_by_id(met)
    
    if met_category == 'protein':
        met_ropacus_coefficient = protein_multiplier*met_subtilis_coefficient
    elif met_category == 'lipid':
        met_ropacus_coefficient = lipid_multiplier*met_subtilis_coefficient
    elif met_category == 'carbohydrate':
        met_ropacus_coefficient = carbohydrate_multiplier*met_subtilis_coefficient
    else:
        met_ropacus_coefficient = met_subtilis_coefficient
        
    metabolites_to_add[metabolite] = met_ropacus_coefficient
    
glucose_growth_equation.add_metabolites(metabolites_to_add)
model.add_reactions([glucose_growth_equation])

model.reactions.get_by_id('Growth_Glucose').reaction

'0.00021600000000000002 10fthf_c + 0.3060466418024825 ala__L_c + 0.00021600000000000002 amet_c + 0.17622738016510175 arg__L_c + 0.14361638594790055 asn__L_c + 0.14361638594790055 asp__L_c + 52.547151 atp_c + 0.005053 ca2_c + 0.005053 cl_c + 0.000559 coa_c + 9.7e-05 cobalt2_c + 0.129616 ctp_c + 0.000688 cu2_c + 0.05456194100309699 cys__L_c + 0.025403 datp_c + 0.026229000000000002 dctp_c + 0.026229000000000002 dgtp_c + 0.025403 dttp_c + 0.00021600000000000002 fad_c + 0.006519 fe2_c + 0.00758 fe3_c + 0.15678572668106583 gln__L_c + 0.15678572668106583 glu__L_c + 0.36499815849357253 gly_c + 0.0013320421171023411 gtca1_45_BS_c + 0.0013320421171023411 gtca2_45_BS_c + 0.0013320421171023411 gtca3_45_BS_c + 0.208826 gtp_c + 47.184845 h2o_c + 0.05644283705841627 his__L_c + 0.1730915306242595 ile__L_c + 0.189503 k_c + 0.26841828831993364 leu__L_c + 6.721942712462895e-05 lipo1_24_BS_c + 6.721942712462895e-05 lipo2_24_BS_c + 6.721942712462895e-05 lipo3_24_BS_c + 6.721942712462895e-05 lipo4_24_BS_c +

### Test glucose biomass equation

Calculate breakdown of macromolecules in mg / g dry cell weight

In [13]:
# Code

### Add glucose biomass equation to model

### Save model with custom biomass equations

In [14]:
cobra.io.write_sbml_model(model, "../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml")
model

Name,ropacus_annotated_curated
Memory address,0x07f6381773110
Number of metabolites,1583
Number of reactions,2385
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"
